In [149]:
import os
import numpy as np
import pandas as pd
from sklearn import svm, model_selection, \
    impute, base, pipeline, preprocessing, \
    compose, ensemble
import matplotlib.pyplot as plt
%matplotlib inline

# Data Prep

In [19]:
HOUSING_PATH = os.path.join("datasets", "housing")

In [20]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [21]:
housing = load_housing_data()

In [22]:
bins = [0, 1.5, 3, 4.5, 6, np.inf]
labels = range(len(bins) - 1)
housing["income_cat"] = pd.cut(housing["median_income"], bins=bins, labels=labels)

In [23]:
kwargs = {
    "n_splits": 1,
    "test_size": 0.2,
    "random_state": 42
}
splitter = model_selection.StratifiedShuffleSplit(**kwargs)
splits = splitter.split(housing, housing["income_cat"])

In [24]:
for train_ix, test_ix in splits:
    train_raw = housing.loc[train_ix]
    test = housing.loc[test_ix]

In [25]:
train_raw = train_raw.drop("income_cat", axis=1)
test_raw = test.drop("income_cat", axis=1)

In [26]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

In [59]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        args = [X, rooms_per_household, population_per_household]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
    def get_added_ftrs(self):
        ret = ["rooms_per_hh", "pop_per_hh"]
        if self.add_bedrooms_per_room:
            ret.append("beds_per_room")
        return ret

In [60]:
tgt = "median_house_value"

train_raw_X = train_raw.drop(tgt, axis=1)
train_y = train_raw[tgt]

test_raw_X = test_raw.drop(tgt, axis=1)
test_y = test_raw[tgt]

In [61]:
num_pipeline = pipeline.Pipeline([
    ("imputer", impute.SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scalar", preprocessing.StandardScaler())
])

cat_ftrs = ["ocean_proximity"]
num_ftrs = [c for c in train_raw_X.columns if c not in cat_ftrs]

full_pipe = compose.ColumnTransformer([
    ("num", num_pipeline, num_ftrs),
    ("cat", preprocessing.OneHotEncoder(), cat_ftrs)
])

In [62]:
train_X = full_pipe.fit_transform(train_raw_X)

## 1.

In [15]:
svr = svm.SVR()

scoring = "neg_mean_squared_error"
scores = model_selection.cross_val_score(svr, train_X, train_y,
                                        scoring=scoring, cv=10)
rmse_scores = np.sqrt(-scores)

In [17]:
print(f"RMSE of SVR: {round(rmse_scores.mean())}")

RMSE of SVR: 118573.0


In [24]:
param_grid = [
    {"kernel": ["linear"], "C": [.1, 1, 10]},
    {"kernel": ["rbf"], "C": [.1, 1, 10]}
]
svr = svm.SVR()
scoring = "neg_mean_squared_error"
grid_search = model_selection.GridSearchCV(svr, param_grid,
                                          scoring=scoring,
                                          cv=5,
                                           return_train_score=True)
grid_search.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.1, 1, 10], 'kernel': ['linear']},
                         {'C': [0.1, 1, 10], 'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [28]:
grid_search.best_estimator_

SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [43]:
scores = grid_search.cv_results_["mean_test_score"]
rmse_scores = np.sqrt(-scores)
print(f"RMSE: {round(rmse_scores.mean())}")

RMSE: 111520.0


## 2.

In [31]:
param_grid = [
    {"kernel": ["linear"], "C": [.1, 1, 10]},
    {"kernel": ["rbf"], "C": [.1, 1, 10]}
]
svr = svm.SVR()
scoring = "neg_mean_squared_error"
n_iter = 1  # Due to runtime considerations
random_search = model_selection.RandomizedSearchCV(svr, param_grid,
                                                   scoring=scoring,
                                                   cv=5, n_iter=n_iter)
random_search.fit(train_X, train_y)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='scale', kernel='rbf',
                                 max_iter=-1, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='deprecated', n_iter=1, n_jobs=None,
                   param_distributions=[{'C': [0.1, 1, 10],
                                         'kernel': ['linear']},
                                        {'C': [0.1, 1, 10], 'kernel': ['rbf']}],
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='neg_mean_squared_error',
                   verbose=0)

In [33]:
scores = random_search.cv_results_["mean_test_score"]
rmse_scores = np.sqrt(-scores)
print(f"RMSE: {round(rmse_scores.mean())}")

RMSE: 116122.0


## 3.

In [37]:
# Use a model that gives an indication of 
# feature importance to get the list of features
forest = ensemble.RandomForestRegressor()
forest.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [80]:
num_ftrs = full_pipe.transformers_[0][2]
cat_ftrs = full_pipe.named_transformers_["cat"].categories_[0].tolist()
added_ftrs = full_pipe.named_transformers_["num"].named_steps["attribs_adder"].get_added_ftrs()
all_ftrs = num_ftrs + cat_ftrs + added_ftrs

In [86]:
ftr_imp = list(zip(all_ftrs, forest.feature_importances_))
print("===FEATURE IMPORTANCES===")
for ftr, imp in sorted(ftr_imp, key=lambda x: x[1], reverse=True):
    print(f"{ftr}: {round(imp, 3)}")

===FEATURE IMPORTANCES===
median_income: 0.472
NEAR OCEAN: 0.141
INLAND: 0.122
longitude: 0.059
latitude: 0.056
housing_median_age: 0.045
<1H OCEAN: 0.028
ISLAND: 0.023
total_rooms: 0.013
households: 0.013
total_bedrooms: 0.012
population: 0.012
beds_per_room: 0.002
NEAR BAY: 0.001
pop_per_hh: 0.001
rooms_per_hh: 0.0


In [89]:
# Take just the top 3
top_n = 3
top_ftrs = sorted(ftr_imp, key=lambda x: x[1], reverse=True)[:top_n]
top_ftrs = [x[0] for x in top_ftrs]

In [103]:
ixs = [all_ftrs.index(ftr) for ftr in top_ftrs]

In [113]:
# Cutting some corners on hardcoded-ness and state reachout
class SpecificFeatureExtrator(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, ixs=ixs):
        if isinstance(ixs, int):
            ixs = [ixs]
        self.ixs = ixs
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_proj = X[:, self.ixs]
        return X_proj

In [117]:
extractor_pipe = pipeline.Pipeline([
    ("trafo", full_pipe),
    ("picker", SpecificFeatureExtrator(ixs=ixs)),
])
train_X_new = extractor_pipe.fit_transform(train_raw_X)

In [119]:
forest = ensemble.RandomForestRegressor()

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [120]:
scoring = "neg_mean_squared_error"
scores = model_selection.cross_val_score(forest, train_X_new, train_y,
                                         scoring=scoring, cv=5)

In [125]:
rmse_scores = np.sqrt(-scores)
print(f"RMSE: {round(rmse_scores.mean())}")

RMSE: 70941.0


# 4.

In [161]:
# A pipeline that does everything,
# i.e. transformation and prediction
model = ensemble.RandomForestRegressor()
end2end = pipeline.Pipeline([
    ("trafo", full_pipe),
    ("est", model)
])

In [146]:
end2end.fit(train_raw_X, train_y)

Pipeline(memory=None,
         steps=[('transformations',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                          

In [147]:
predictions = end2end.predict(test_raw_X)

In [148]:
rmse = np.sqrt(np.mean((test_y - predictions) ** 2))
rmse

48518.59031044192

# 5.

In [172]:
scoring = "neg_mean_squared_error"
param_grid = [
    {"trafo__num__imputer__strategy": ["median", "mean"]}
]
grid_search = model_selection.GridSearchCV(end2end, param_grid,
                                           scoring=scoring, cv=5)
grid_search.fit(train_raw_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('trafo',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                           

In [183]:
grid_search.best_estimator_.get_params()["trafo__num__imputer"].get_params()["strategy"]

'median'